## function-calling介绍

根据OpenAI官方文档，function-calling是使得大型语言模型具备可以连接到外部工具的能力。简而言之，开发者事先给模型提供了若干工具（函数），在模型理解用户的问题后，自行判断是否需要调用工具以获得更多上下文信息，帮助模型更好的决策。

## 用途

function-calling有什么用？OpenAI官网给出3个例子：

- 创建通过调用外部 API 回答问题的助手
- 将自然语言转换为 API 调用
- 从文本中提取结构化数据

## 定义function

首先定义4个function，这4个function有不同的使用场景。

In [ ]:
def get_gas_prices(city: str) -> float:
    """Get gas prices for specified cities."""
    print(f'Get gas prices for {city}.')


def github(project: str) -> str:
    '''Get the infomations such as author from github with the project name.'''
    print(f'Access the github for {project}.')


def get_weather(city: str) -> str:
    """Get the current weather given a city."""
    print(f'Getting weather for {city}.')


def get_directions(start: str, destination: str) -> float:
    """Get directions from Google Directions API.
    start: start address as a string including zipcode (if any)
    destination: end address as a string including zipcode (if any)"""
    print(f'Get directions for {start} {destination}.')

## 定义Prompts

In [ ]:
functions_prompt = f"""
    You have access to the following tools:
    {function_to_json(get_weather)}
    {function_to_json(get_gas_prices)}
    {function_to_json(get_directions)}
    {function_to_json(github)}
    
    You must follow these instructions:
    Always select one or more of the above tools based on the user query
    If a tool is found, you must respond in the JSON format matching the following schema:
    {{
       "tools": {{
            "tool": "<name of the selected tool>",
            "tool_input":<parameters for the selected tool, matching the tool's JSON schema>
       }}
    }}
    If there are multiple tools required, make sure a list of tools are returned in a JSON array.
    If there is no tool that match the user request, you must respond empty JSON {{}}.
    
    User Query:
    """ 

这是一个复杂的提示，让我们一步一步来看：

首先，告诉模型我提供了4个工具，让它自己去查询这4个工具的元数据。function_to_json函数返回的内容如下：
```python
{
  "name": "get_weather",
  "description": "Get the current weather given a city.",
  "parameters": {
    "type": "object",
    "properties": {
      "city": {
        "type": "str"
      }
    }
  },
  "returns": "str"
}
```

这一步就是让模型根据这几个函数的元数据来理解函数，尤其是description写的应该尽量详细。

第二步提示模型作出自己的判断，根据上面提供的工具选择一个或多个进行调用，并指定了模型返回数据格式——JSON以及这个JSON的schema。

第三步等待用户的问题。

## 完整代码

In [9]:
import inspect
import json
import requests
from typing import get_type_hints

def generate_full_completion(model: str, prompt: str) -> dict[str, str]:
    params = {"model": model, "prompt": prompt, "stream": False}
    response = requests.post(
        "http://localhost:11434/api/generate",
        headers={"Content-Type": "application/json"},
        data=json.dumps(params),
        timeout=60,
    )
    return json.loads(response.text)

def get_gas_prices(city: str) -> float:
    """Get gas prices for specified cities."""
    print(f'Get gas prices for {city}.')

def github(project: str) -> str:
    '''Get the infomations such as author from github with the project name.'''
    print(f'Access the github for {project}.')

def get_weather(city: str) -> str:
    """Get the current weather given a city."""
    print(f'Getting weather for {city}.')

def get_directions(start: str, destination: str) -> float:
    """Get directions from Google Directions API.
    start: start address as a string including zipcode (if any)
    destination: end address as a string including zipcode (if any)"""
    print(f'Get directions for {start} {destination}.')

def get_type_name(t):
    name = str(t)
    if "list" in name or "dict" in name:
        return name
    else:
        return t.__name__

def function_to_json(func):
    signature = inspect.signature(func)
    type_hints = get_type_hints(func)

    function_info = {
        "name": func.__name__,
        "description": func.__doc__,
        "parameters": {"type": "object", "properties": {}},
        "returns": type_hints.get("return", "void").__name__,
    }

    for name, _ in signature.parameters.items():
        param_type = get_type_name(type_hints.get(name, type(None)))
        function_info["parameters"]["properties"][name] = {"type": param_type}

    return json.dumps(function_info, indent=2)

def main():
    functions_prompt = f"""
    You have access to the following tools:
    {function_to_json(get_weather)}
    {function_to_json(get_gas_prices)}
    {function_to_json(get_directions)}
    {function_to_json(github)}

    You must follow these instructions:
    Always select one or more of the above tools based on the user query
    If a tool is found, you must respond in the JSON format matching the following schema:
   {{   "tools": {{        "tool": "<name of the selected tool>",
        "tool_input": <parameters for the selected tool, matching the tool's JSON schema>
    }}}}
    If there are multiple tools required, make sure a list of tools are returned in a JSON array.
    If there is no tool that match the user request, you must respond empty JSON {{}}.
    User Query:
    """
    GPT_MODEL = "mistral:7b-instruct-v0.2-q8_0"
    # select model: https://ollama.com/search?c=tools
    # 1. mistral:7b-instruct-v0.2-q8_0
    
    prompts = [
    "What's the weather like in Beijing?",
    "What is the distance from Shanghai to Hangzhou and how much do I need to fill up the gas in advance to drive from Shanghai to Hangzhou?",
    "Who's the author of the 'snake-game' on github?",
    "What is the exchange rate between US dollar and Japanese yen?",
    ]
    for prompt in prompts:
        print(f"❓{prompt}")
        question = functions_prompt + prompt
        response = generate_full_completion(GPT_MODEL, question)
        try:
            data = json.loads(response.get("response", response))
            # print(data)
            for tool_data in data["tools"]:
                execute_fuc(tool_data)
            print(f"> Total duration: {int(response.get('total_duration')) / 1e9} seconds")
        except Exception:
            print('No tools found.')
            print(f"> Total duration: {int(response.get('total_duration')) / 1e9} seconds")

def execute_fuc(tool_data):
    func_name = tool_data["tool"]
    func_input = tool_data["tool_input"]
    # 获取全局命名空间中的函数对象
    func = globals().get(func_name)
    if func is not None and callable(func):
    # 如果找到了函数并且是可调用的，调用它
        func(**func_input)
    else:
        print(f"Unknown function: {func_name}")
if __name__ == "__main__":
    main()

❓What's the weather like in Beijing?
Getting weather for Beijing.
> Total duration: 2.757826337 seconds
❓What is the distance from Shanghai to Hangzhou and how much do I need to fill up the gas in advance to drive from Shanghai to Hangzhou?
Get directions for Shanghai, China Hangzhou, China.
Get gas prices for Shanghai.
> Total duration: 1.302087687 seconds
❓Who's the author of the 'snake-game' on github?
Access the github for snake-game.
> Total duration: 0.665892936 seconds
❓What is the exchange rate between US dollar and Japanese yen?
No tools found.
> Total duration: 0.74020361 seconds


整段代码就是用来测试模型对问题的理解能力以及是否能正确判断调用哪个工具的。

我们来看下设定的4个问题：

1. "What's the weather like in Beijing?",
2. "What is the distance from Shanghai to Hangzhou and how much do I need to fill up the gas in advance to drive from Shanghai to Hangzhou?",
3. "Who's the author of the 'snake-game' on github?",
4. "What is the exchange rate between US dollar and Japanese yen?",

按照我们的设想，如果模型理解了我提供的工具的功能以及读懂了用户的问题，应该按照以下规则选择工具：

1. 问题1应该对应的是`get_weather('Beijing')`
2. 问题2应该对应的是`get_directions('Shanghai', 'Hangzhou')`和`get_gas_prices('Shanghai')`
3. 问题3应该对应的是`github('snake-game')`
4. 问题4应该没有对应的函数，模型不选择任何工具

## 官方实现

In [11]:
import ollama

# select model from https://ollama.com/search?c=tools
response = ollama.chat(
    model='llama3.1',
    messages=[{'role': 'user', 'content': 
        'What is the weather in Toronto?'}],

	# provide a weather checking tool to the model
    tools=[{
      'type': 'function',
      'function': {
        'name': 'get_current_weather',
        'description': 'Get the current weather for a city',
        'parameters': {
          'type': 'object',
          'properties': {
            'city': {
              'type': 'string',
              'description': 'The name of the city',
            },
          },
          'required': ['city'],
        },
      },
    },
  ],
)

print(response['message']['tool_calls'])

[{'function': {'name': 'get_current_weather', 'arguments': {'city': 'Toronto'}}}]


In [23]:
import openai
import json

openai.base_url = "http://localhost:11434/v1/"
openai.api_key = 'ollama'

messages=[{'role': 'user', 'content': 'What is the weather in Toronto?'}]

tools=[{
      'type': 'function',
      'function': {
        'name': 'get_current_weather',
        'description': 'Get the current weather for a city',
        'parameters': {
          'type': 'object',
          'properties': {
            'city': {
              'type': 'string',
              'description': 'The name of the city',
            },
          },
          'required': ['city'],
        },
      },
    },
  ]

def get_current_weather(city):
    return "\033[32m"+city+" is good!\033[0m"

response = openai.chat.completions.create(
	model="llama3.1",
	messages=messages,
	tools=tools,
)
print(response)
print(response.choices[0].message.tool_calls[0])
print(response.choices[0].message.tool_calls[0].function)
print(response.choices[0].message.tool_calls[0].function.arguments)
print(json.loads(response.choices[0].message.tool_calls[0].function.arguments)["city"])
print(response.choices[0].message.tool_calls[0].function.name)

if response.choices[0].message.tool_calls[0].function.name=="get_current_weather":
    city_weather = get_current_weather(json.loads(response.choices[0].message.tool_calls[0].function.arguments)["city"])
    print(city_weather)

ChatCompletion(id='chatcmpl-576', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_isxscrll', function=Function(arguments='{"city":"Toronto"}', name='get_current_weather'), type='function')]))], created=1729211825, model='llama3.1', object='chat.completion', service_tier=None, system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tokens=18, prompt_tokens=168, total_tokens=186, completion_tokens_details=None, prompt_tokens_details=None))
ChatCompletionMessageToolCall(id='call_isxscrll', function=Function(arguments='{"city":"Toronto"}', name='get_current_weather'), type='function')
Function(arguments='{"city":"Toronto"}', name='get_current_weather')
{"city":"Toronto"}
Toronto
get_current_weather
Toronto is good!


## 异步执行

In [25]:
import json
import ollama
import asyncio


# Simulates an API call to get flight times
# In a real application, this would fetch data from a live database or API
def get_flight_times(departure: str, arrival: str) -> str:
  flights = {
    'NYC-LAX': {'departure': '08:00 AM', 'arrival': '11:30 AM', 'duration': '5h 30m'},
    'LAX-NYC': {'departure': '02:00 PM', 'arrival': '10:30 PM', 'duration': '5h 30m'},
    'LHR-JFK': {'departure': '10:00 AM', 'arrival': '01:00 PM', 'duration': '8h 00m'},
    'JFK-LHR': {'departure': '09:00 PM', 'arrival': '09:00 AM', 'duration': '7h 00m'},
    'CDG-DXB': {'departure': '11:00 AM', 'arrival': '08:00 PM', 'duration': '6h 00m'},
    'DXB-CDG': {'departure': '03:00 AM', 'arrival': '07:30 AM', 'duration': '7h 30m'},
  }

  key = f'{departure}-{arrival}'.upper()
  return json.dumps(flights.get(key, {'error': 'Flight not found'}))


async def run(model: str):
  client = ollama.AsyncClient()
  # Initialize conversation with a user query
  messages = [{'role': 'user', 'content': 'What is the flight time from New York (NYC) to Los Angeles (LAX)?'}]

  # First API call: Send the query and function description to the model
  response = await client.chat(
    model=model,
    messages=messages,
    tools=[
      {
        'type': 'function',
        'function': {
          'name': 'get_flight_times',
          'description': 'Get the flight times between two cities',
          'parameters': {
            'type': 'object',
            'properties': {
              'departure': {
                'type': 'string',
                'description': 'The departure city (airport code)',
              },
              'arrival': {
                'type': 'string',
                'description': 'The arrival city (airport code)',
              },
            },
            'required': ['departure', 'arrival'],
          },
        },
      },
    ],
  )

  # Add the model's response to the conversation history
  messages.append(response['message'])

  # Check if the model decided to use the provided function
  if not response['message'].get('tool_calls'):
    print("The model didn't use the function. Its response was:")
    print(response['message']['content'])
    return

  # Process function calls made by the model
  if response['message'].get('tool_calls'):
    available_functions = {
      'get_flight_times': get_flight_times,
    }
    for tool in response['message']['tool_calls']:
      function_to_call = available_functions[tool['function']['name']]
      function_response = function_to_call(tool['function']['arguments']['departure'], tool['function']['arguments']['arrival'])
      # Add function response to the conversation
      messages.append(
        {
          'role': 'tool',
          'content': function_response,
        }
      )

  # Second API call: Get final response from the model
  final_response = await client.chat(model=model, messages=messages)
  print(final_response['message']['content'])


# Run the async function
#asyncio.run(run('mistral'))
await run('mistral')

 The flight time from New York (NYC) to Los Angeles (LAX) is approximately 5 hours and 30 minutes. The departure is at 8:00 AM, and the arrival is at 11:30 AM (EST).
